<a href="https://colab.research.google.com/github/superjaru/Web-Scraping-Tris-Rating-/blob/main/TrisScrapingBySelenium_(Using_debian_buster).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap
apt remove chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'chromium-browser' is not installed, so not removed
Package 'chromium-browser-l10n' is not installed, so not removed
Package 'chromium-codecs-ffmpeg-extra' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Executing: /tmp/apt-key-gpghome.ugeUJm8I1f/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.BX8AqZN8tJ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.UoxaFoFgts/gpg.1.

In [ ]:
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd


def pagesource(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("start-maximized")
    options.headless = True
    # chrome_options.add_argument('--disable-dev-shm-usage')
    # chrome_options.add_argument('start-maximized')
    driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)  # If get stuck here , cancel and go run again
    driver.get(url)
    user = driver.find_element(By.ID,"uName")
    user.send_keys("another.jaru@gmail.com")
    passwrd = driver.find_element(By.XPATH, "//input[@name='uPassword'][@class='ccm-input-text']")
    passwrd.send_keys("Bbl123")
    submitBtn = driver.find_element(By.ID,"submit")
    driver.execute_script("arguments[0].click();",submitBtn)
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'page-detail')))
        print("Page is ready!")
    except:
        print('HTML element not found!!')
    row_data=[]
    header = []
    linksOfPDF = []
    condition = True
    page = 1
    while condition :
      driver.get("https://www.trisrating.com/rating-information/rating-list/?ccm_paging_p=" + str(page))
      soup = BeautifulSoup(driver.page_source)
      table_rating = soup.find('table')
      company = soup.find('tbody', {'id':'page-detail'})
      for row in table_rating('tr'):
        col = row.find_all('td')
        col = [ele.text.strip() for ele in col]
        row_data.append(col)
      a_tags = company.find_all('a')
      for e in a_tags:
        # href_main = e.find_all('href')
        link = 'https://www.trisrating.com/index.php' + e['href']
        driver.get(link)
        soup2 = BeautifulSoup(driver.page_source)
        company2= soup2.find('tr', {'class':'each-row'})
        if company2 is None :
          linksOfPDF.append('None')
        else :
          a_tags2 = company2.find_all('a')
          for e in a_tags2:
            # href_main2 = e.find('href')
            links2 = 'https://www.trisrating.com' + e['href']
            linksOfPDF.append(links2)
      pg = soup.find('div','ccm-pagination')
      active_pg = pg.find('span',{'class':'currentPage active numbers'})
      if not active_pg.findNext('span',{'class':' numbers'}):
        condition = False
      else :
        page += 1
        # print('page counttt', page)
      # if page == 15:
      #   break
    for i in soup.find_all("th"):
      col_name = i.text.strip().lower().replace(" ","_")
      header.append(col_name)
    allCompany = list(filter(None,row_data)) 
    df = pd.DataFrame(allCompany, columns = header)
    df.insert(7,"File pdf",linksOfPDF , True)
    df.to_excel('Tris_Rating 09-01-2023.xlsx', sheet_name = 'Tris_Rating', index=False)
    driver.close()
    return   

url = "https://www.trisrating.com/index.php/login/do_login/";
pagesource(url)


<ipython-input-2-28e8ba285f49>:20: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
<ipython-input-2-28e8ba285f49>:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)  # If get stuck here , cancel and go run again


SessionNotCreatedException: ignored